In [79]:
import requests
import pandas as pd
import numpy as np
import random

In [ ]:
api_key = ''

In [16]:
url_flights = 'https://api.aviationstack.com/v1/flights'
params_flights = {
    'access_key':api_key,
    'airline_iata':'UA'
}

api_result_flights = requests.get(url_flights, params_flights)

In [ ]:
# Pull 1000 flights
flights = []
for i in range(11):
    print(i*100)
    offset = i*100
    params_flights = {
        'access_key':api_key,
        'airline_iata':'UA',
        'offset':offset
    }

    api_result_flights = requests.get(url_flights, params_flights)
    api_response_flights = api_result_flights.json()
    flights.append(api_response_flights['data'])


0
100
200
300
400
500
600
700
800
900
1000


In [173]:
# Combine all flight data into a single object
flights_long = []
for i, call in enumerate(flights[0:]):
    flights_long = flights_long + call

# flatten
flights_flat = pd.json_normalize(flights_long, sep='_')
flights_flat['departure_delayed'] = np.where(flights_flat['departure_delay'] > 0, 1, 0)

# Synthesize delay codes
codes = ["WO", "WT", "WR", "AT", "FC", "FP"]
def synth_delay_code(delay):
    if delay == 1:
        return np.random.choice(codes)
    else:
        return None

flights_flat['departure_delay_code'] = flights_flat['departure_delayed'].apply(synth_delay_code)

In [176]:
empty_seats = pd.DataFrame([random.randint(0, 15) for _ in range(len(flights_flat))], columns=['empty_seats'])

In [179]:
df = pd.concat([flights_flat, empty_seats], axis=1)

keep = ['flight_date', 'airline_name', 'airline_iata', 'flight_iata', 'empty_seats',
    'flight_status', 'departure_delayed', 'departure_delay_code', 
    'departure_airport', 'departure_iata', 'departure_terminal', 'departure_gate', 
    'departure_delay', 'departure_scheduled', 'departure_estimated', 'departure_actual',
    'arrival_airport', 'arrival_iata', 'arrival_icao', 'arrival_terminal', 'arrival_gate',
    'arrival_scheduled', 'arrival_delay', 'arrival_estimated', 'arrival_actual']

# Remove international origin flights
df_clean = df[keep][df['departure_icao'].str.contains('^K|^PA|^PO|^PP')]

In [ ]:
df_clean.to_csv('../data/flights.csv')

In [171]:
df_clean.head()

,flight_date,airline_name,airline_iata,flight_iata,empty_seats,flight_status,departure_delayed,departure_delay_code,departure_airport,departure_iata,...,departure_actual,arrival_airport,arrival_iata,arrival_icao,arrival_terminal,arrival_gate,arrival_scheduled,arrival_delay,arrival_estimated,arrival_actual
68,2025-06-20,United Airlines,UA,UA384,9,active,1,FC,San Francisco International,SFO,...,None,Honolulu International,HNL,PHNL,None,None,2025-06-20T15:04:00+00:00,NaN,None,None
69,2025-06-20,United Airlines,UA,UA2264,7,scheduled,0,None,Washington Dulles International,IAD,...,None,Portland International,PDX,KPDX,3,E3,2025-06-20T19:52:00+00:00,NaN,None,None
70,2025-06-20,United Airlines,UA,UA1431,8,scheduled,1,FC,Los Angeles International,LAX,...,None,Honolulu International,HNL,PHNL,2,None,2025-06-20T16:01:00+00:00,NaN,None,None
71,2025-06-20,United Airlines,UA,UA6616,6,active,1,FC,Philadelphia International,PHL,...,2025-06-20T15:34:00+00:00,Pierre Elliott Trudeau International,YUL,CYUL,None,77,2025-06-20T17:00:00+00:00,NaN,None,None
73,2025-06-20,United Airlines,UA,UA4900,4,scheduled,1,FC,George Bush Intercontinental,IAH,...,None,Mc Ghee Tyson,TYS,KTYS,None,None,2025-06-20T19:56:00+00:00,NaN,None,None


In [172]:
df_clean[['departure_delay', 'departure_delayed', 'departure_delay_code']]

,departure_delay,departure_delayed,departure_delay_code
68,226.0,1,FC
69,NaN,0,None
70,18.0,1,FC
71,15.0,1,FC
73,21.0,1,FC
...,...,...,...
1093,12.0,1,FC
1094,NaN,0,None
1095,18.0,1,FC
1096,17.0,1,FC
